In [2]:
import MKE as dp
import numpy as np
import Graphics as gr

# Example 1
Solving a simple overdetermined system. Supports are named "a", "b",... from left to right. Beams are named "first", "second",... from let to right.
<img src="files/overdetermined1.jpg" style="width: 500px;"/>

In [3]:
a = dp.Support("a", 'fixed', 0, 0)
b = dp.Support("b", 'endpoint', 1, 0)
c = dp.Support("c", "pinned", 2, 0)

first = dp.Beam('first', [a,], [b,],  0.006, 200e6, 0.01, 8000, 30, 0)
second = dp.Beam('second', [b,], [c,],  0.006, 200e6, 0.01, 8000, 30, 0)





first.transformation_matrix()
first.form_elements()
first.form_M()
first.form_K()

second.transformation_matrix()
second.form_elements()
second.form_M()
second.form_K()

b.set_boundryforces(0, 0, 0, 0, 1000, 0)

beams = (first, second)
supports = (a, b, c)


In [4]:
#Assemble equations

M, b1, Naming, M_ = dp.form_mainMatrix(beams, 
                                       supports, dynamic=True)
            
    
#Perform calculčations

r = np.linalg.solve(M, b1)
R, Naming_new = dp.resoults_parsing(beams, 
                                    supports, r, Naming)

NF, eigenvectors = dp.natural_frequencies(M_, M)
NF = np.sort(NF)
NF[:5]

first.calculate_forces(R[0])
second.calculate_forces(R[1])

In [5]:
def info():
    print("===============================================")
    print('{0:10}  {1:10}'.format("L", "Loading"))
    print('{0:10}  {1:10}'.format("u", "Axial deformation"))
    print('{0:10}  {1:10}'.format("w", "Deflection"))
    print('{0:10}  {1:10}'.format("N", "Tension"))
    print('{0:10}  {1:10}'.format("T", "Shear"))
    print('{0:10}  {1:10}'.format("M", "Bending Moment"))
    print('{0:10}  {1:10}'.format("TAB", "Legend"))
    print('{0:10}  {1:10}'.format("ESC", "Quit graphical"))
    print("===============================================")

info()

def draw1():
    global draw
    draw = gr.Drawing()
    draw.scaling(beams)
    for beam_ in beams:
        draw.draw_beam(beam_)
    for sup_ in supports:
        draw.draw_support(sup_)

u_, w_, N_, T_, M_, q_, n_ = [], [], [], [], [], [], []

for beam in beams:
    u_ = np.hstack((u_, beam.u))
    w_ = np.hstack((w_, beam.w))
    q_ = np.hstack((q_, np.array(beam.q_).flatten()))
    n_ = np.hstack((n_, np.array(beam.n_).flatten()))
    try:
        N_ = np.hstack((N_, beam.N))
        T_ = np.hstack((T_, beam.T))
        M_ = np.hstack((M_, beam.M))
    except:
        pass
u_ = np.hstack((u_, 0))
w_ = np.hstack((w_, 0))
n_ = np.hstack((n_, 0))
q_ = np.hstack((q_, 0))
N_ = np.hstack((N_, 0))
T_ = np.hstack((T_, 0))
M_ = np.hstack((M_, 0))

size_u = max(u_) - min(u_)
size_w = max(w_) - min(w_)
size_n = max(n_) - min(n_)
size_q = max(q_) - min(q_)
size_N = max(N_) - min(N_)
size_T = max(T_) - min(T_)
size_M = max(M_) - min(M_)

F_ = []
for sup in (supports):
    F_.append(np.sqrt(sup.Fx**2 + sup.Fy**2))
    F_.append(0)

size_F = max(F_) - min(F_)

draw1()
u, w, T, N, M, l, Legend = False, False, False, False, False, False, False
running = True
while running:
    for event in gr.pygame.event.get():
        if event.type == gr.pygame.QUIT or event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_ESCAPE:
            running = False
            print("KONEC")
        if event.type == gr.pygame.KEYDOWN:
            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_s:
                gr.pygame.image.save(draw.screen, "Structural_analysis.jpg")
            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_u:
                u = ~u

            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_l:
                l = ~l

            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_w:
                w = ~w

            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_t:
                T = ~T

            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_n:
                N = ~N

            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_m:
                M = ~M
            if event.type == gr.pygame.KEYDOWN and event.key == gr.pygame.K_TAB:
                Legend = ~Legend

            draw1()

            if l:
                for beam in beams:
                    if size_q:
                        size = (max(np.hstack((beam.q_, 0))) - min(np.hstack((beam.q_, 0)))) / size_q * 100
                        draw.draw_loading(beam.q_, beam, 1, draw.RED, draw_size=size)
                    if size_n:
                        size = (max(beam.n_) - min(beam.n_)) / size_n * 100
                        draw.draw_loading(beam.n_, beam, -1, draw.BLACK, draw_size=size)

                for sup in supports:
                    print(size_F)
                    if size_F:
                        size = (sup.Fx**2 + sup.Fy**2)/size_F**2
                    else:
                        size = 0
                    draw.draw_boundryloading(sup, size*100, 20)

            if u:
                for beam in beams:
                    size = (max(np.hstack((beam.u, 0))) - min(np.hstack((beam.u, 0))))/size_u * 100
                    draw.draw_loading(beam.u, beam, 1, draw.BLUE, draw_size = size)
            if w:
                for beam in beams:
                    size = (max(np.hstack((beam.w, 0))) - min(np.hstack((beam.w, 0))))/size_w * 100
                    draw.draw_loading(beam.w, beam, -1, draw.GREEN, draw_size = size)
            if N:
                for beam in beams:
                    size = (max(np.hstack((beam.N, 0))) - min(np.hstack((beam.N, 0))))/size_N * 100
                    draw.draw_loading(beam.N, beam, 1, draw.DARKOLIVEGREEN, draw_size = size)
            if T:
                for beam in beams:
                    size = (max(np.hstack((beam.T, 0))) - min(np.hstack((beam.T, 0))))/size_T * 100
                    draw.draw_loading(beam.T, beam, -1, draw.LBLUE, draw_size = size)
            if M:
                for beam in beams:
                    size = (max(np.hstack((beam.M, 0))) - min(np.hstack((beam.M, 0))))/size_M * 100
                    draw.draw_loading(beam.M, beam, 1, draw.PURPLE, draw_size = size)
            if Legend:
                draw.legend()
        gr.pygame.display.update()

L           Loading   


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:120: RuntimeWarning: invalid value encountered in double_scalars


u           Axial deformation
w           Deflection
N           Tension   
T           Shear     
M           Bending Moment
TAB         Legend    
ESC         Quit graphical
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
1000.0
KONEC
1000.0
1000.0
1000.0
